[![Generic badge](https://img.shields.io/badge/License-Properitary-red.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/blob/master/LICENSE.md) <br>
[![Active Development](https://img.shields.io/badge/Maintenance%20Level-Actively%20Developed-brightgreen.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/) <br>
[![Ask Me Anything !](https://img.shields.io/badge/Ask%20me-anything-1abc9c.svg)](https://github.com/JanGaida/research_project_machine_learning_hshof_sose2020/issues) <br>
[![Python 3.7.7](https://img.shields.io/badge/python-3.7.7-blue.svg)](https://www.python.org/downloads/release/python-377/)

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Logo_fh_hof.svg/2000px-Logo_fh_hof.svg.png" width="300" style="background-color:#FFF">

<font size="+4"><i><u>Q-Learning mit 'Super Mario Bros'</u></i></font>

Seminararbeit der Vorlesung **Angewandtes Maschinelles Lernen** an der **Hochschule für angewande Wissenschaften Hof** des **Sommersemesters 2020**.

# Vorbereitungen

*Code der das Jupyter-Notebook vorbereit.*

## Notebook-Styling

*Ändert u.a. das Layout des Notebooks.*

In [1]:
from IPython.core.display import display, HTML

_notebook_relative_width = "80%" # Wertebereich: 0-100, muss nach HTML-Schema definiert sein
display(HTML('<style>.container { width: ' + _notebook_relative_width + ' !important; }</style>'))

## Requirements

*Die benötigten Packages wurden aus der bei mir installierten Pip-Liste gebildet.*

**Hinweis:** *Es werden* ***mehr als zwingend notwendige*** *Packages installiert.*

In [2]:
# Package-Liste
#!pip list

In [3]:
%%capture
# Installation
#!pip3 install -r requirements.txt

In [4]:
%%capture
# Update 
#!pip3 list --outdated --format=freeze | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U

In [5]:
%%capture
# Export Pip-Packages nach './requirements.txt'
!pip3 freeze > requirements.txt

# Konfiguration

*Import-Statements & Vorbereitung der Frameworks/Module zusammengefasst.*

## Allgemein

*Nicht weiter spezifizierte Import-Statements, bspw. von System-Bibliotheken.*

In [6]:
import os
from datetime import datetime

In [7]:
# Erstelle Logs-Folder falls nicht existent.
logs_base_dir = "./logs"
if not os.path.isdir(logs_base_dir):
    !mkdir logs

## Tensorflow

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1280px-TensorFlowLogo.svg.png" width="150px"/>

*TensorFlow ist ein Framework zur datenstromorientierten Programmierung. Es wird aus Python-Programmen heraus benutzt und ist in Python und C++ implementiert. Populäre Anwendung findet TensorFlow im Bereich des maschinellen Lernens. [...]*

[Weiter Informationen](https://pypi.org/project/tensorflow/)

In [8]:
%%capture
import tensorflow as tf

## Tensorboard

<img src="https://www.tensorflow.org/site-assets/images/project-logos/tensorboard-logo-social.png" width="250px"/>

*TensorBoard ist ein Sammlung von Web-Applicationen um TensorFlow zu inspizieren und zu verstehen.*

[Weitere Informationen](https://pypi.org/project/tensorboard/)

In [9]:
%load_ext tensorboard

In [10]:
%%capture
from tensorboard import notebook
# Starten einer Tensorboard-Instanz mit Logs & gebindeten Ports
%tensorboard --port=6007 --logdir {logs_base_dir} --bind_all
# Auflisten der TensorBoard-Instanzen (resourcen intesiv)
notebook.list()

## IPyWidgets

In [11]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## PyVirtualDisplay
*Ein python wrapper für Xvfb, Xephyr und Xvnc.*

[Weiter Informationen](https://pypi.org/project/PyVirtualDisplay/)

In [12]:
# videocode dependecies; x11-utils als workaround für 'xdpyinfo'
print("In case of failure in the next Code-Block consider installing python-opengl, xvfb, ffmpeg, x11-utils - Packages via apt-get.\nUse command 'sudo apt-get install -y python-opengl xvfb ffmpeg x11-utils > /dev/null' inside a terminal.")

In case of failure in the next Code-Block consider installing python-opengl, xvfb, ffmpeg, x11-utils - Packages via apt-get.
Use command 'sudo apt-get install -y python-opengl xvfb ffmpeg x11-utils > /dev/null' inside a terminal.


In [13]:
from pyvirtualdisplay import Display as PyVDisplay

global record_base_dir

record_base_dir = "./mp4"
if not os.path.isdir(record_base_dir):
    os.mkdir(record_base_dir)

py_vdisplay = PyVDisplay(visible=0, size=(1400, 900))
py_vdisplay.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1257'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1257'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

## OpenAI-Gym

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/OpenAI_Logo.svg/1920px-OpenAI_Logo.svg.png" width="275px"/>

*OpenAI-Gym ist ein Toolkit für das Entwickeln und Vergleichen von Reinforcement-Learning-Algorithmen.*

[Weitere Informationen](https://pypi.org/project/gym/)

In [14]:
import gym

from gym.wrappers import Monitor
from gym import logger as gymlogger

# DEBUG = 10 # INFO = 20 # WARN = 30 # ERROR = 40 # DISABLED = 50
gymlogger.set_level(50)

## Gym-Super-Mario-Bros

*Eine OpenAI-Gym-Umgebung für Super Mario Bros. und Super Mario Bros. 2 (Lost Levels) auf den NES (Nitendo Entertainment System) mithilfe des nes-py emulators.*

[Weitere Informationen](https://pypi.org/project/gym-super-mario-bros/)

<table><thead><tr><th>SuperMarioBros-v0</th><th>SuperMarioBros-v1</th><th>SuperMarioBros-v2</th><th>SuperMarioBros-v3</th><th>SuperMarioBros2-v0</th><th>SuperMarioBros2-v1</th></tr></thead><tbody><tr><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/c4717c633d3823dda390ebc21bac34b18e7c22c3/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832302d33643135653563322d363833302d313165382d383164342d6563666166666565306131342e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/e8eda56caeefcada9af67f43385ef1f48d0ac394/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831392d33636666366334382d363833302d313165382d383337332d3866616431363635616337322e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7f53e40eb716be49673cf41fb833486ab3ee104a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831382d33636561303964342d363833302d313165382d386566612d3866333464386230356231312e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/51975e7cc634efb02ed92acfb56368733b25f4d9/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383831372d33636436363030612d363833302d313165382d386162622d3963656536613331643337372e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/0618011a5c6cedb9dba051b8cf134ba51dd0777a/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832322d33643362383431322d363833302d313165382d383630622d6166333830326635333733662e706e67" width="175"></td><td><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/7c42437f4d2f447e192c088eab22739534c2d9be/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f323138343436392f34303934383832312d33643264363161322d363833302d313165382d383738392d6139326537353061613961382e706e67" width="175"></td></tr></tbody></table>

In [15]:
import gym_super_mario_bros

from nes_py.wrappers import JoypadSpace # nes_py is a direct dependency by gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY, SIMPLE_MOVEMENT, COMPLEX_MOVEMENT

global gym_enviorment_id
global gym_enviorment_movementset

gym_enviorment_id = 'SuperMarioBros-v0'
gym_enviorment_movementset = 'Simple'

@interact
def show_env_param(Enviorment=['SuperMarioBros-v0', 'SuperMarioBros-v1', 'SuperMarioBros-v2', 'SuperMarioBros-v3', 'SuperMarioBros2-v0', 'SuperMarioBros2-v1', 'SuperMarioBros2-v2', 'SuperMarioBros2-v3'],
                   Movementset=['Simple', 'Complex', 'Only-Right']):
    global gym_enviorment_id
    global gym_enviorment_movementset
    
    gym_enviorment_id = Enviorment
    gym_enviorment_movementset = Movementset
    
    return

interactive(children=(Dropdown(description='Enviorment', options=('SuperMarioBros-v0', 'SuperMarioBros-v1', 'S…

## Stable-Baselines

*Stable Baselines ist eine Sammlung von optimierten Implementation von Reinforcement-Learning-Algorithmen basierend auf OpenAi.*

[Weitere Informationen](https://pypi.org/project/stable-baselines/)

In [16]:
from stable_baselines import PPO2

# Pre-Training

*Code der vor dem Traniern ausgeführt wird.*

## Trainings-Parameter

### Trainings-Epochen

In [17]:
print('Geben Sie eine Anzahl an zu trainierenden Epochen ein (Wertebereich: 1-∞; Default: 1).')

max_amount_of_epoches = 1

@interact
def show_articles_more_than(Epochen=''):
    global max_amount_of_epoches
    try:
        tmp = int(Epochen)
        
        if tmp > 0:
            max_amount_of_epoches = tmp
            print('Epochen-Anzahl in Höhe von {} angewandt.'.format(max_amount_of_epoches))
        else:
            max_amount_of_epoches = 1
            print('Default-Anzahl in Höhe von 1 Epochen angewandt.')
    except ValueError:
        max_amount_of_epoches = 1
        print('Default-Anzahl in Höhe von 1 Epochen angewandt.')

Geben Sie eine Anzahl an zu trainierenden Epochen ein (Wertebereich: 1-∞; Default: 1).


interactive(children=(Text(value='', description='Epochen'), Output()), _dom_classes=('widget-interact',))

### Monitor-Funktion

In [18]:
global gym_enviorment_monitor

gym_enviorment_monitor = True

@interact
def show_monitor_param(Mointor_Run = True):
    global gym_enviorment_monitor
    gym_enviorment_monitor = Mointor_Run
    return

interactive(children=(Checkbox(value=True, description='Mointor_Run'), Output()), _dom_classes=('widget-intera…

## Reward-Funktionen

*Belohnungsfunktionen zum lernen.*

# Training

## Helpers

### Env

In [19]:
def init_env():
    global gym_enviorment
    global gym_enviorment_id
    global gym_enviorment_movementset
    
    if gym_enviorment == None:
        # Create new Enviorment
        print("Intializing Gym ...\nApplying Gym-Enviorment-Id \"{}\" ...".format(gym_enviorment_id))
        gym_enviorment = gym.make(gym_enviorment_id)
            
        if gym_enviorment_movementset == 'Complex':
            gym_enviorment = JoypadSpace(gym_enviorment, COMPLEX_MOVEMENT)
        elif gym_enviorment_movementset == 'Simple':
            gym_enviorment = JoypadSpace(gym_enviorment, SIMPLE_MOVEMENT)
        elif gym_enviorment_movementset == "Only-Right":
            gym_enviorment = JoypadSpace(gym_enviorment, RIGHT_ONLY)
        else:
            print("Error: Gym-Movementset unknown ...\nFalling back to 'SIMPLE_MOVEMENT' ...")
            gym_enviorment_movementset = 'Simple'
            gym_enviorment = JoypadSpace(gym_enviorment, SIMPLE_MOVEMENT)
        
        print('Applied Gym-Enviorment-Movementset: {} ...\nIntializing Gym completed ...'.format(gym_enviorment_movementset))
    
    else:
        try:
            print("Reusing Gym ...")
            gym_enviorment.reset()
        except:
            # When reset failed due to gym still beeing active
            print("Reusing Gym failed...")
            gym_enviorment.close()
        finally:
            gym_enviorment = None
            init_env()

### Run

In [20]:
def init_run(cur_amount_of_epoches):
    global gym_enviorment
    global gym_enviorment_monitor
    global gym_enviorment_run_path
    
    print("Initialzing Run-{} ...".format(cur_amount_of_epoches))
    
    init_env()
    
    if gym_enviorment_monitor:
        if not gym_enviorment_run_path == None:
            # Create directory for the recordings of this specific run
            gym_enviorment_monitor_path = "{}/{}".format(gym_enviorment_run_path, cur_amount_of_epoches)
            print("Creating Videooutput-Path \"{}\" ...".format(gym_enviorment_monitor_path))
            if not os.path.isdir(gym_enviorment_monitor_path):
                os.mkdir(gym_enviorment_monitor_path)
            else:
                print("Warning: \"{}\"-Path already exist while it should'nt have ...".format(gym_enviorment_run_path))
            
            # Wrapp with Monitor
            print("Wrapping Enviorment with Monitor ...")
            gym_enviorment = Monitor(gym_enviorment, gym_enviorment_monitor_path, force = False)
            
        else:
            print("Warning: Enviorment_Monitor-Path is empty while it should'nt have ...\nFailed attaching Monitor for run {} ...".format(cur_amount_of_epoches))
    
    gym_enviorment.reset()
    
    print("Initialzing Run-{} completed...".format(cur_amount_of_epoches))

In [21]:
def run_gym(cur_amount_of_epoches):
    
    init_run(cur_amount_of_epoches)

    loop_render()
    
    deinit_run(cur_amount_of_epoches)

In [22]:
def deinit_run(cur_amount_of_epoches):
    global gym_enviorment
    
    print("Deinitialzing Run-{} ...".format(cur_amount_of_epoches))
    
    print("Deinitialzing Run-{} completed...".format(cur_amount_of_epoches))

### Loop

In [23]:
def loop_render():
    global gym_enviorment
    global observation
    
    # Cleanup
    observation = None
    done = None
    info = None
    
    print("Running ...")
    for step in range(2000): # for Debuging
    #while True: # for actual Runs
        
        observation, _, done, info = gym_enviorment.step(gym_enviorment.action_space.sample());
        
        if done:
            observation = gym_enviorment.reset();
            break
        
        gym_enviorment.render();

### Epoch

In [24]:
def init_epoches():
    global max_amount_of_epoches
    global record_base_dir
    global gym_enviorment_run_path
    global gym_enviorment_monitor
    global gym_enviorment
    
    print("Initialzing Epoches-Runs ...")
    
    # Init env var
    if not 'gym_enviorment' in globals():
        gym_enviorment = None
        
    # Prepare Directory
    if gym_enviorment_monitor:
        print("Preparing Epoches-Runs-Directory ...")
        # Create path with format: {record_base_dir}/run_{amount_runs}_{datetime_now}
        gym_enviorment_run_path = '{}/run_{}_{}'.format(record_base_dir, max_amount_of_epoches, datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))
    
        # Ensure unique directory
        if os.path.isdir(gym_enviorment_run_path):
            gym_enviorment_run_path = '{}_uuid-{}'.format(gym_enviorment_run_path, uuid.uuid4())
        
        # Create dir
        os.mkdir(gym_enviorment_run_path)
        print("Epoches-Runs-Directory-Path is \"{}\" ...".format(gym_enviorment_run_path))
    else:
        gym_enviorment_run_path = None
        
    print("Initialzing Epoches-Runs completed...")

In [25]:
def run_gym_epoches():
    global gym_enviorment_run_path
    global max_amount_of_epoches
    
    print("Preparing ...\nApplied maximum of {} Epoches ...".format(max_amount_of_epoches))
    init_epoches()
    
    print("Starting Epoches-Loop ...\n")
    cur_amount_of_epoches = 1
    while cur_amount_of_epoches <= max_amount_of_epoches:
        print("\nRunning Gym {}/{} ...\n".format(cur_amount_of_epoches, max_amount_of_epoches))
        run_gym(cur_amount_of_epoches)
        print('Finished Gym ...')
        cur_amount_of_epoches += 1
    
    deinit_epoches()
    print("Finished Epoches ...\n")

In [26]:
def deinit_epoches():
    global gym_enviorment
    
    print("Deinitialzing Epoches ...")
    
    if not gym_enviorment == None:
        gym_enviorment.close()
        gym_enviorment = None
    
    print("Deinitialzing completed...")

## Start

Call-Reihenfolge ist:

```
run_gym_epoches
|> init_epoches
|      run_gym
|      |> init_run
|      |      |> init_env
|      |> loop_render
|      |> deinit_run
|> deinit_epoches
```

In [27]:
run_gym_epoches()

Preparing ...
Applied maximum of 1 Epoches ...
Initialzing Epoches-Runs ...
Preparing Epoches-Runs-Directory ...
Epoches-Runs-Directory-Path is "./mp4/run_1_25-04-2020_19-37-21" ...
Initialzing Epoches-Runs completed...
Starting Epoches-Loop ...


Running Gym 1/1 ...

Initialzing Run-1 ...
Intializing Gym ...
Applying Gym-Enviorment-Id "SuperMarioBros-v0" ...
Applied Gym-Enviorment-Movementset: Simple ...
Intializing Gym completed ...
Creating Videooutput-Path "./mp4/run_1_25-04-2020_19-37-21/1" ...
Wrapping Enviorment with Monitor ...
Initialzing Run-1 completed...
Running ...


/home/jan/anaconda3/lib/python3.7/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Deinitialzing Run-1 ...
Deinitialzing Run-1 completed...
Finished Gym ...
Deinitialzing Epoches ...
Deinitialzing completed...
Finished Epoches ...



Exception ignored in: <function Monitor.__del__ at 0x7f5802e17560>
Traceback (most recent call last):
  File "/home/jan/anaconda3/lib/python3.7/site-packages/gym/wrappers/monitor.py", line 229, in __del__
    self.close()
  File "/home/jan/anaconda3/lib/python3.7/site-packages/gym/wrappers/monitor.py", line 134, in close
    super(Monitor, self).close()
  File "/home/jan/anaconda3/lib/python3.7/site-packages/gym/core.py", line 238, in close
    return self.env.close()
  File "/home/jan/anaconda3/lib/python3.7/site-packages/gym/core.py", line 238, in close
    return self.env.close()
  File "/home/jan/anaconda3/lib/python3.7/site-packages/gym/core.py", line 238, in close
    return self.env.close()
  File "/home/jan/anaconda3/lib/python3.7/site-packages/nes_py/nes_env.py", line 338, in close
    raise ValueError('env has already been closed.')
ValueError: env has already been closed.
